In [163]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.technical import ma
import yfinance as yf
from pandas.tseries.offsets import BMonthEnd
import math
import alpaca_trade_api as tradeapi


In [164]:
data = yf.download("AAPL", start="1999-05-01", end="2020-10-04")
data.to_csv("spy.csv")

[*********************100%***********************]  1 of 1 completed


In [165]:
class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        super(MyStrategy, self).__init__(feed)
        self.instrument = instrument
        self.sma_200 = ma.SMA(feed["spy"].getAdjCloseDataSeries(),200)
        self.position=None
        self.count = 0
        
    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        #self.info(f"BUY {execInfo.getQuantity()} shares at ${execInfo.getPrice()})")

    def onEnterCanceled(self, position):
        self.position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        #self.info(f"SELL {execInfo.getQuantity()} shares at ${execInfo.getPrice()})")
        self.position = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.position.exitMarket()
    
    def onBars(self, bars):
        offset = BMonthEnd()
        SMA = self.sma_200[-1]
        ADJ_CLOSE = bars.getBar(self.instrument).getAdjClose()
        CLOSE = bars.getBar(self.instrument).getClose()
        DATE = bars.getBar(self.instrument).getDateTime()
        last_day=offset.rollforward(DATE)
        BROKER = self.getBroker()
        CASH = BROKER.getCash()

        if(DATE == last_day):
            if(SMA is not None):
                if(ADJ_CLOSE>SMA and self.position is None):
                    print(f"Cash: {self.getBroker().getCash()}, Equity: {self.getBroker().getEquity()}")
                    
                    # set quantity to buy at 95% of our ability to buy. I learned that the trade actually executes the next trading day so we have to account for minor changes
                    quantity = int(((self.getBroker().getCash()/CLOSE)*0.95))
                    
                    print(f"Buy on {DATE.date()}: {quantity} shares at {CLOSE:,.2f}. 200 day SMA is [{SMA:,.2f}].")
                    
                    #enter long position. ~95% of buying power used
                    self.position = self.enterLong(self.instrument, quantity, goodTillCanceled=True, allOrNone=False)
                    
                elif(ADJ_CLOSE<SMA and not self.position is None):
                    # metrics for this position as we close it
                    roi = self.position.getReturn()
                    quantity = self.position.getShares()
                    self.count += 1
                    
                    # exit position
                    self.position.exitMarket(True)
                    
                    # print statements, no logic here
                    print(f"Sell on {DATE.date()}: {quantity} shares at {CLOSE}. 200 day SMA is [{SMA:,.2f}].")
                    print(f"ROI: {roi*100}")
                    print(f"Portfolio Value: {self.getBroker().getEquity()}")
                    print(f"Trade Count: {self.count}")
                    print(f"\n \n******************** \n")
            
        

# Load the bar feed from the CSV file
feed = yahoofeed.Feed()
feed.addBarsFromCSV("spy", "spy.csv")

# Evaluate the strategy with the feed's bars.
myStrategyInstance = MyStrategy(feed, "spy")
myStrategyInstance.run()




Cash: 1000000, Equity: 1000000
Buy on 2000-02-29: 928243 shares at 0.88. 200 day SMA is [0.59].
Sell on 2000-05-31: 928243 shares at 0.75. 200 day SMA is [0.75].
ROI: -29.151307469597853
Portfolio Value: 713549.7904083729
Trade Count: 1

 
******************** 

Cash: 694902.3440149426, Equity: 694902.3440149426
Buy on 2000-06-30: 705848 shares at 0.80. 200 day SMA is [0.78].
Sell on 2000-07-31: 705848 shares at 0.9073659777641296. 200 day SMA is [0.81].
ROI: -2.5180420458111077
Portfolio Value: 678358.6528685689
Trade Count: 2

 
******************** 

Cash: 672056.841270268, Equity: 672056.841270268
Buy on 2000-08-31: 586722 shares at 0.94. 200 day SMA is [0.83].
Sell on 2000-09-29: 586722 shares at 0.4598209857940674. 200 day SMA is [0.84].
ROI: -58.002077870720385
Portfolio Value: 299461.9385202527
Trade Count: 3

 
******************** 

Cash: 309284.85047471523, Equity: 309284.85047471523
Buy on 2001-06-29: 707696 shares at 0.36. 200 day SMA is [0.34].
Sell on 2001-07-31: 707696 